# 文本生成

## 说明
本文展示了如何使用appbuilder内置组件以及如何快速构建全新的文本生成组件。以下是一个在营销场景下生成各种文本内容的样例。

## 概览
具体地，本样例按照以下流程进行文本生成：
1. 使用**空模板**构建一个全新的文本生成组件：商品信息生成组件。
2. 输入一个商品，使用新构建的商品信息生成组件生成该商品的商品信息。
3. 基于生成的商品信息，使用内置的**问答对挖掘**组件生成问答对。生成的问答对可用于客服等。
4. 基于生成的商品信息，使用内置的**风格写作**组件为该商品生成一份【小红书】文案。
5. 使用内置的**标签抽取**组件对生成的文案进行标签抽取。抽取出的标签可用于检索文案。

以下我们会演示如何实现我们自己的定制化能力。

## 流程演示
### Step 0: 安装Python SDK
我们的appbuilder支持使用pip安装（要求Python >= 3.8）：

In [ ]:
pip install appbuilder-sdk

代码中需要配置用户的`APPBUILDER_TOKEN`。

In [2]:
import os

# 设置环境变量
os.environ['APPBUILDER_TOKEN'] = '...'

安装成功后，我们就可以搭建我们自己的能力了。

### Step 1: 商品信息生成
基于appbuilder提供的**空模板**，用户可以自定义文本生成组件以解决多样化的需求。这里我们构建一个**商品信息生成**组件以生成商品信息：

In [ ]:
from appbuilder import Message, Playground

# 输入到大模型中的prompt的模板
prompt_template = \
'''输入商品名，我需要你为我生成该商品的商品信息。

要求：
- 你生成的商品信息应该包含多方面信息。

商品名：{product_name}
商品信息：
'''
# 创建商品信息生成组件
product_information_generation = Playground(prompt_template=prompt_template, model='eb-4')

# 获取商品信息
# 填充prompt_template参数的参数映射表，需要与prompt_template对应
prompt_template_kwargs = {
    'product_name': '特斯拉Model Y'
}
response = product_information_generation(Message(prompt_template_kwargs), stream=False, temperature=0.5)
product_information = response.content
print(f'商品信息：\n{product_information}')

输出结果：

```
商品信息：
特斯拉Model Y是一款全电动SUV，具有出色的性能、安全和智能科技特点。以下是该商品的详细信息：

1. 车型概述：特斯拉Model Y是一款中型豪华SUV，结合了特斯拉的电动车技术和现代化设计。它提供了宽敞的内部空间、强大的动力和令人印象深刻的续航里程。
2. 外观设计：特斯拉Model Y的外观设计简洁而现代，具有流线型车身和独特的LED大灯组。车顶的全景玻璃天窗为乘客提供了极佳的视野。
3. 内饰设计：内饰方面，特斯拉Model Y采用了极简的设计风格，中控台配备了一个大尺寸触摸屏，用于控制车辆的各种功能。座椅舒适，后排座椅可按比例放倒，增加储物空间。
4. 动力系统：特斯拉Model Y提供了多种动力选择，包括后轮驱动和全轮驱动配置。其电动机可产生惊人的扭矩，加速迅猛。根据配置不同，最大续航里程可达到数百公里。
5. 充电和续航：特斯拉Model Y支持特斯拉超级充电站和家庭充电桩进行充电。在特斯拉超级充电站，车辆可以在短时间内快速充满电。
6. 智能科技：特斯拉Model Y配备了特斯拉自家的Autopilot自动驾驶辅助系统，可以实现自动驾驶、自动泊车、车道保持等功能。此外，车辆还支持OTA在线升级，不断获得新功能和改进。
7. 安全性能：特斯拉Model Y在安全性能方面表现出色，配备了多项主动安全技术，如碰撞预警、自动紧急制动、盲点监测等。同时，车身结构坚固，为乘客提供良好的保护。
8. 价格和购买方式：特斯拉Model Y的价格根据车型和配置的不同而异。您可以通过特斯拉官方网站或授权经销商进行购买。
```

### Step 2: 问答对生成
基于Step 1生成的商品信息我们生成一些问答对，这里我们使用内置的**问答对生成**组件。

In [ ]:
from appbuilder import QAPairMining

# 初始化问答对生成组件
qa_pair_mining = QAPairMining(model='eb-turbo-appbuilder')

# 获取问答对
response = qa_pair_mining(Message(product_information), stream=False, temperature=1e-10)
qa_pairs = response.content
print(f'问答对：\n{qa_pairs}')

输出结果：

```
问答对：
问题：特斯拉Model Y是一款什么车型？
答案：特斯拉Model Y是一款全电动SUV。

问题：特斯拉Model Y的外观设计如何？
答案：特斯拉Model Y的外观设计简洁而现代，具有流线型车身和独特的LED大灯组。车顶的全景玻璃天窗为乘客提供了极佳的视野。

问题：特斯拉Model Y的内饰设计风格是什么？
答案：内饰方面，特斯拉Model Y采用了极简的设计风格，中控台配备了一个大尺寸触摸屏，用于控制车辆的各种功能。座椅舒适，后排座椅可按比例放倒，增加储物空间。

问题：特斯拉Model Y提供了哪些动力选择？
答案：特斯拉Model Y提供了多种动力选择，包括后轮驱动和全轮驱动配置。其电动机可产生惊人的扭矩，加速迅猛。根据配置不同，最大续航里程可达到数百公里。

问题：特斯拉Model Y的充电方式有哪些？
答案：特斯拉Model Y支持特斯拉超级充电站和家庭充电桩进行充电。在特斯拉超级充电站，车辆可以在短时间内快速充满电。

问题：特斯拉Model Y配备了哪些智能科技？
答案：特斯拉Model Y配备了特斯拉自家的Autopilot自动驾驶辅助系统，可以实现自动驾驶、自动泊车、车道保持等功能。此外，车辆还支持OTA在线升级，不断获得新功能和改进。

问题：特斯拉Model Y在安全性能方面有哪些特点？
答案：特斯拉Model Y在安全性能方面表现出色，配备了多项主动安全技术，如碰撞预警、自动紧急制动、盲点监测等。同时，车身结构坚固，为乘客提供良好的保护。
```

### Step 3: 营销文案生成
基于Step 1生成的商品信息生成【小红书】文案，这里我们使用内置的**风格写作**组件。

In [ ]:
from appbuilder import StyleWriting

# 构造query
query = \
f'''请你基于以下商品信息生成文案：
{product_information}
'''

# 初始化风格写作组件
style_writing = StyleWriting(model='eb-4')

# 获取小红书文案
response = style_writing(Message(query), style_query='小红书', length=300)
copywriting = response.content
print(f'文案：\n{copywriting}')

输出结果：

```
文案：
【特斯拉Model Y：电动之美，智能之魅】

🚗 探寻未来驾驶的乐趣，特斯拉Model Y带你领略电动之美！流线型车身、独特LED大灯组🔦，展现现代豪华SUV的风采。而车内极简设计、大尺寸触摸屏🖥️，尽显科技感。

💥 动力强劲，后驱或全驱配置任君选择，扭矩惊人，加速迅猛！续航里程更是达到数百公里，让你畅享无忧出行🔋。

🔌 充电便捷，特斯拉超级充电站和家庭充电桩让你随时随地为爱车补给能量。短时间内快速充满电，不再为充电烦恼💡。

🎉 智能科技满满！Autopilot自动驾驶辅助系统实现自动驾驶、自动泊车等功能，让驾驶更轻松。OTA在线升级让你随时拥有最新功能和改进✨。

🛡️ 安全性能卓越，多项主动安全技术为你保驾护航。碰撞预警、自动紧急制动等让你驾驶更安心。坚固车身结构为乘客提供良好保护👨‍👩‍👧‍👦。

特斯拉Model Y，不仅是一款电动车，更是智能科技的杰作！快来体验吧！#特斯拉ModelY #电动车 #智能科技 #豪华SUV #未来驾驶
```

### Step 4: 标签抽取
基于Step 3生成的文案进行标签抽取，这里我们使用内置的**标签抽取**组件。

In [ ]:
from appbuilder import TagExtraction

# 初始化标签抽取组件
tag_extraction = TagExtraction(model='eb-turbo-appbuilder')

# 获取标签
response = tag_extraction(Message(copywriting), stream=False, temperature=1e-10)
tags = response.content
print(f'标签：\n{tags}')

输出结果：

```
1.特斯拉ModelY
2.电动之美
3.智能之魅
4.动力强劲
5.续航里程
6.充电便捷
7.自动驾驶
8.OTA在线升级
9.安全性能
10.保护乘客
```